In [52]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.pipeline import make_pipeline, make_union
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from tpot import *
from xgboost import *
from sklearn.ensemble import *
from sklearn.svm import *
import warnings
warnings.filterwarnings('ignore')

# DATASET : Owner median < 50 k

In [2]:
df = pd.read_csv('../raw_data/data_final_indé_medium.csv')

In [3]:
X = df.drop(columns=['steam_appid', 'name', 'top_5_tags','score_descriptif','owner_estimated'])

In [4]:
y = df['owner_estimated']

#### Train Test Split


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

#### Stacking

In [6]:
forest = RandomForestRegressor(
                               bootstrap=True, 
                               max_features=0.4, 
                               min_samples_leaf=14, 
                               n_estimators=100,
                               min_samples_split=14)

linear = LinearSVR(C=0.01, 
                  dual=False, 
                  epsilon=0.001, 
                  loss='squared_epsilon_insensitive', 
                  tol=0.0001)

xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)

estimators = [
    ('lr', forest),
    ('svr', LinearSVR(random_state=42)),
    ('xgb', xgb)
]


ensemble = StackingRegressor(
    estimators = estimators,
    final_estimator = LinearRegression())

ensemble.fit(X_train, y_train)

print(ensemble.score(X_test,y_test))

0.32477463965301767


#### Test Tpot 

In [45]:

# instanciate TPOTClassifier
tpot = TPOTRegressor(generations=4, population_size=50, verbosity=2, scoring='r2', n_jobs=-1, cv=2)
# process autoML with TPOT
tpot.fit(X_train, y_train)

# print score
print(tpot.score(X_test, y_test))


Optimization Progress:   0%|          | 0/250 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.3102517799058878

Generation 2 - Current best internal CV score: 0.3102517799058878

Generation 3 - Current best internal CV score: 0.3176604457150901

Generation 4 - Current best internal CV score: 0.3176604457150901

Best pipeline: XGBRegressor(input_matrix, learning_rate=0.1, max_depth=4, min_child_weight=2, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.55, verbosity=0)
0.3185798754451563


## TEST avec une nouvelle feature : Ratio 

In [12]:
dfb = pd.read_csv('../raw_data/data_final_indé_medium2.csv')

In [28]:
Xb = dfb.drop(columns=['steam_appid', 'name', 'top_5_tags','score_descriptif','owner_estimated','sells_per_days'])

In [29]:
Xb.columns

Index(['topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
       'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11',
       'topic_12', 'topic_13', 'topic_14', 'topic_15', 'topic_16', 'topic_17',
       'topic_18', 'topic_19', 'english', 'price', 'days_on_steam',
       'is_a_remake', 'has_a_website', 'ratio'],
      dtype='object')

In [30]:
yb = dfb['owner_estimated']

#### Train test split

In [31]:
Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb, test_size = 0.3, random_state = 0)

#### Stacking


In [37]:
forest = RandomForestRegressor(
                               bootstrap=True, 
                               max_features=0.4, 
                               min_samples_leaf=14, 
                               n_estimators=100,
                               min_samples_split=14)

linear = LinearSVR(C=0.01, 
                  dual=False, 
                  epsilon=0.001, 
                  loss='squared_epsilon_insensitive', 
                  tol=0.0001)

xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)

estimators = [
    ('lr', forest),
    ('svr', LinearSVR(random_state=42)),
    ('xgb', xgb)
]


ensemble = StackingRegressor(
    estimators = estimators,
    final_estimator = LinearRegression())

scores = cross_val_score(ensemble, Xb, yb, cv=5) # Cross validate

print(scores)

<built-in method mean of numpy.ndarray object at 0x17be3c8a0>


In [39]:
scores.mean()

0.29379261503045634

In [41]:
forest = RandomForestRegressor(
                               bootstrap=True, 
                               max_features=0.4, 
                               min_samples_leaf=14, 
                               n_estimators=100,
                               min_samples_split=14)

linear = LinearSVR(C=0.01, 
                  dual=False, 
                  epsilon=0.001, 
                  loss='squared_epsilon_insensitive', 
                  tol=0.0001)

xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)

estimators = [
    ('lr', forest),
    ('svr', LinearSVR(random_state=42)),
    ('xgb', xgb)
]


ensemble = StackingRegressor(
    estimators = estimators,
    final_estimator = LinearRegression())

cv_results = cross_validate(ensemble, Xb, yb, cv=5)

# Scores
cv_results['test_score']

# Mean of scores
cv_results['test_score'].mean()

#ensemble.fit(Xb_train, yb_train)

#print(ensemble.score(Xb_test,yb_test))

0.29414893564521793

In [44]:
# instanciate TPOTRegressor
tpot = TPOTRegressor(generations=4, population_size=50, verbosity=2, scoring='r2', n_jobs=-1, cv=5)
# process autoML with TPOT
tpot.fit(Xb_train, yb_train)

# print score
print(tpot.score(Xb_test, yb_test))

Optimization Progress:   0%|          | 0/250 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.4102701823869929

Generation 2 - Current best internal CV score: 0.4102701823869929


/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")
/Users/mathieudbt/.pyenv/versions/3.8.12


Generation 3 - Current best internal CV score: 0.42524659918590124


/Users/mathieudbt/.pyenv/versions/3.8.12/envs/steamator/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")



Generation 4 - Current best internal CV score: 0.42524659918590124

Best pipeline: XGBRegressor(CombineDFs(input_matrix, CombineDFs(input_matrix, input_matrix)), learning_rate=0.1, max_depth=10, min_child_weight=19, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.5, verbosity=0)
0.43027783426771715


## TEST modèle en enlevant les jeux < 1 €


In [53]:
dfb = dfb[dfb['price']>= 1]

In [59]:
Xb_ = dfb.drop(columns=['steam_appid', 'name', 'top_5_tags','score_descriptif','owner_estimated','sells_per_days'])


In [60]:
yb_ = dfb['owner_estimated']

In [61]:
Xb__train, Xb__test, yb__train, yb__test = train_test_split(Xb_, yb_, test_size = 0.3, random_state = 0)

In [62]:
forest = RandomForestRegressor(
                               bootstrap=True, 
                               max_features=0.4, 
                               min_samples_leaf=14, 
                               n_estimators=100,
                               min_samples_split=14)

linear = LinearSVR(C=0.01, 
                  dual=False, 
                  epsilon=0.001, 
                  loss='squared_epsilon_insensitive', 
                  tol=0.0001)

xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)

estimators = [
    ('lr', forest),
    ('svr', LinearSVR(random_state=42)),
    ('xgb', xgb)
]


ensemble = StackingRegressor(
    estimators = estimators,
    final_estimator = LinearRegression())

scores = cross_val_score(ensemble, Xb_, yb_, cv=5) # Cross validate

print(scores.mean())

0.2710309544045145


In [58]:
'''
# instanciate TPOTRegressor
tpot = TPOTRegressor(generations=4, population_size=50, verbosity=2, scoring='r2', n_jobs=-1, cv=5)
# process autoML with TPOT
tpot.fit(Xb__train, yb__train)

# print score
print(tpot.score(Xb__test, yb__test))
'''

"\n# instanciate TPOTRegressor\ntpot = TPOTRegressor(generations=4, population_size=50, verbosity=2, scoring='r2', n_jobs=-1, cv=5)\n# process autoML with TPOT\ntpot.fit(Xb__train, yb__train)\n\n# print score\nprint(tpot.score(Xb__test, yb__test))\n"

## TEST avec deux nouvelles features supplémentaires : Sales per day per main topic / pondération sales per topic

In [63]:
bf = pd.read_csv('../raw_data/data_final_indé_medium3.csv')

In [65]:
bf.columns

Index(['steam_appid', 'name', 'top_5_tags', 'topic_0', 'topic_1', 'topic_2',
       'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8',
       'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14',
       'topic_15', 'topic_16', 'topic_17', 'topic_18', 'topic_19', 'english',
       'price', 'days_on_steam', 'is_a_remake', 'has_a_website',
       'owner_estimated', 'score_descriptif', 'ratio', 'sells_per_days',
       'sells_per_days_per_maintopic', 'ponderation_sales_per_topics'],
      dtype='object')

In [67]:
X_bf = bf.drop(columns=['steam_appid', 'name', 'top_5_tags','score_descriptif','owner_estimated','sells_per_days'])

In [68]:
y_bf = bf['owner_estimated']

In [69]:
Xbf_train, Xbf_test, ybf_train, ybf_test = train_test_split(X_bf, y_bf, test_size = 0.3, random_state = 0)

In [70]:
forest = RandomForestRegressor(
                               bootstrap=True, 
                               max_features=0.4, 
                               min_samples_leaf=14, 
                               n_estimators=100,
                               min_samples_split=14)

linear = LinearSVR(C=0.01, 
                  dual=False, 
                  epsilon=0.001, 
                  loss='squared_epsilon_insensitive', 
                  tol=0.0001)

xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)

estimators = [
    ('lr', forest),
    ('svr', LinearSVR(random_state=42)),
    ('xgb', xgb)
]


ensemble = StackingRegressor(
    estimators = estimators,
    final_estimator = LinearRegression())

scores = cross_val_score(ensemble, X_bf, y_bf, cv=5) # Cross validate

print(scores)

[0.06746292 0.22142316 0.34059417 0.40458509 0.3549247 ]


In [71]:
scores.mean()

0.2777980078308755

In [77]:
ensemble.fit(Xbf_train, ybf_train)

StackingRegressor(estimators=[('lr',
                               RandomForestRegressor(max_features=0.4,
                                                     min_samples_leaf=14,
                                                     min_samples_split=14)),
                              ('svr', LinearSVR(random_state=42)),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            enable_categorical=False,
                                            gamma=None, gpu_id=None,
                                            importance_type=None,
                                            interaction_...
                                            learning_rate=0.1,
                               

In [79]:
ow_df = ensemble.predict(Xbf_test)

In [80]:
o_df = pd.DataFrame(ow_df)

In [90]:
o_df

,0
0,5882.796921
1,4532.611826
2,22529.603256
3,23148.952952
4,1970.193602
...,...
6328,7854.898699
6329,2977.389412
6330,3905.443638
6331,5174.636960


In [92]:
bf.corr().style.background_gradient(cmap='coolwarm',text_color_threshold=0.600)

,steam_appid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,english,price,days_on_steam,is_a_remake,has_a_website,owner_estimated,score_descriptif,ratio,sells_per_days,sells_per_days_per_maintopic,ponderation_sales_per_topics
steam_appid,1.000000,-0.062089,-0.005379,-0.026634,0.077689,-0.049775,0.065331,-0.045358,-0.099821,-0.024673,-0.049202,0.073012,-0.030931,0.042454,-0.047937,-0.039995,-0.033745,0.028924,-0.078848,-0.048579,0.134360,-0.105714,-0.148908,-0.868111,nan,-0.294889,-0.404527,-0.119326,0.020260,0.152456,-0.054807,-0.069291
topic_0,-0.062089,1.000000,-0.048407,-0.039218,-0.064725,-0.034693,-0.060681,-0.034169,-0.048625,-0.055493,-0.058006,-0.051224,-0.053897,-0.068508,-0.046687,-0.039755,-0.054388,-0.070531,-0.039410,-0.062890,-0.117437,0.031831,-0.032488,0.044353,nan,0.021569,0.061057,0.010792,0.033579,0.005024,0.036176,0.049508
topic_1,-0.005379,-0.048407,1.000000,-0.025923,-0.041723,-0.024067,-0.047218,-0.029663,-0.040384,-0.045757,-0.037646,-0.060262,-0.027649,-0.031838,-0.034203,-0.007863,-0.043998,-0.054267,-0.035984,-0.041463,-0.083112,0.028103,-0.043209,0.025046,nan,-0.009054,0.000634,-0.075721,0.056536,-0.018845,-0.074738,-0.068910
topic_2,-0.026634,-0.039218,-0.025923,1.000000,-0.058469,-0.006503,-0.061011,-0.024152,-0.047401,-0.043633,-0.049239,-0.053023,-0.028323,-0.047033,-0.034097,-0.015305,-0.034300,-0.066090,-0.033623,-0.035247,-0.109558,0.029660,-0.012314,0.002829,nan,0.034965,0.008206,0.028388,0.069261,-0.007116,-0.005711,-0.005827
topic_3,0.077689,-0.064725,-0.041723,-0.058469,1.000000,-0.057122,-0.058491,-0.028139,-0.054987,-0.050257,-0.060205,-0.012887,-0.049006,-0.038916,-0.047350,-0.031193,-0.055070,-0.054697,-0.022114,-0.056492,-0.112946,-0.106812,0.051968,-0.070260,nan,-0.037509,0.095580,-0.002432,0.061411,0.111945,0.501516,0.547071
topic_4,-0.049775,-0.034693,-0.024067,-0.006503,-0.057122,1.000000,-0.050879,-0.024635,-0.043596,-0.036942,-0.032247,-0.006787,-0.035887,-0.036239,-0.018478,-0.033138,-0.035630,-0.063998,-0.029014,-0.033523,-0.109121,0.027440,0.043761,0.022513,nan,0.041200,0.107792,0.079665,-0.005954,0.067130,0.235397,0.258904
topic_5,0.065331,-0.060681,-0.047218,-0.061011,-0.058491,-0.050879,1.000000,-0.052591,-0.045183,-0.050914,-0.057489,-0.063185,-0.050870,-0.074923,-0.051706,-0.048761,-0.065658,-0.065379,-0.045863,-0.066010,-0.103905,-0.039054,-0.015233,-0.067113,nan,0.020932,-0.060256,0.052800,-0.022492,0.006062,0.050700,0.049997
topic_6,-0.045358,-0.034169,-0.029663,-0.024152,-0.028139,-0.024635,-0.052591,1.000000,-0.044904,-0.035759,-0.028967,-0.036736,-0.035073,-0.044531,-0.030702,-0.013810,-0.031750,-0.049142,-0.020012,-0.025980,-0.095836,0.017378,0.058089,0.012732,nan,0.045432,0.103465,0.075293,-0.003941,0.081726,0.383872,0.407950
topic_7,-0.099821,-0.048625,-0.040384,-0.047401,-0.054987,-0.043596,-0.045183,-0.044904,1.000000,-0.042966,-0.043497,-0.048920,-0.041001,-0.050862,-0.037303,-0.007438,-0.045338,-0.063336,-0.011108,-0.052003,-0.106673,0.024169,0.018795,0.116384,nan,0.047808,0.021910,-0.097711,0.012334,-0.026341,-0.134124,-0.136253
topic_8,-0.024673,-0.055493,-0.045757,-0.043633,-0.050257,-0.036942,-0.050914,-0.035759,-0.042966,1.000000,-0.032665,-0.044330,-0.025079,-0.046721,-0.037683,-0.028854,-0.041863,-0.051516,-0.039540,-0.037226,-0.101013,-0.003095,0.049169,0.038307,nan,-0.004772,-0.022139,-0.013672,-0.049829,-0.022959,-0.103377,-0.112690
